## Importing Libraries

In [1]:
import findspark
findspark.init()
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.\
        builder.\
        appName('Hyundai_SP').\
        getOrCreate()

## Preparing the data

In [3]:
df = spark.read.csv('cruise_ship_info.csv', inferSchema = True, header = True)

In [4]:
df.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)



In [5]:
for item in df.head(1)[0]:
    print(item)

Journey
Azamara
6
30.276999999999997
6.94
5.94
3.55
42.64
3.55


### Encoding the Categorical Data

In [6]:
df.groupBy('Cruise_line').count().show()

+-----------------+-----+
|      Cruise_line|count|
+-----------------+-----+
|            Costa|   11|
|              P&O|    6|
|           Cunard|    3|
|Regent_Seven_Seas|    5|
|              MSC|    8|
|         Carnival|   22|
|          Crystal|    2|
|           Orient|    1|
|         Princess|   17|
|        Silversea|    4|
|         Seabourn|    3|
| Holland_American|   14|
|         Windstar|    3|
|           Disney|    2|
|        Norwegian|   13|
|          Oceania|    3|
|          Azamara|    2|
|        Celebrity|   10|
|             Star|    6|
|  Royal_Caribbean|   23|
+-----------------+-----+



In [7]:
from pyspark.ml.feature import StringIndexer
encoder = StringIndexer(inputCol = 'Cruise_line', outputCol = 'Cruise_line_encoded').fit(df)
output = encoder.transform(df)
output.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)
 |-- Cruise_line_encoded: double (nullable = false)



### Assembling the Feature Vector for ML Model

In [8]:
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols = ['Age', 'Tonnage', 'passengers', 'length', 
                                        'cabins', 'passenger_density', 'Cruise_line_encoded'], 
                            outputCol = 'features')
output = assembler.transform(output)
output.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)
 |-- Cruise_line_encoded: double (nullable = false)
 |-- features: vector (nullable = true)



In [9]:
final_data = output.select('features', 'crew')
final_data.show()

+--------------------+----+
|            features|crew|
+--------------------+----+
|[6.0,30.276999999...|3.55|
|[6.0,30.276999999...|3.55|
|[26.0,47.262,14.8...| 6.7|
|[11.0,110.0,29.74...|19.1|
|[17.0,101.353,26....|10.0|
|[22.0,70.367,20.5...| 9.2|
|[15.0,70.367,20.5...| 9.2|
|[23.0,70.367,20.5...| 9.2|
|[19.0,70.367,20.5...| 9.2|
|[6.0,110.23899999...|11.5|
|[10.0,110.0,29.74...|11.6|
|[28.0,46.052,14.5...| 6.6|
|[18.0,70.367,20.5...| 9.2|
|[17.0,70.367,20.5...| 9.2|
|[11.0,86.0,21.24,...| 9.3|
|[8.0,110.0,29.74,...|11.6|
|[9.0,88.5,21.24,9...|10.3|
|[15.0,70.367,20.5...| 9.2|
|[12.0,88.5,21.24,...| 9.3|
|[20.0,70.367,20.5...| 9.2|
+--------------------+----+
only showing top 20 rows



### Splitting the data into train and test data

In [10]:
train_data, test_data = final_data.randomSplit([0.7, 0.3])

In [11]:
train_data.describe().show()

+-------+------------------+
|summary|              crew|
+-------+------------------+
|  count|               120|
|   mean|  7.77775000000001|
| stddev|3.5716341533267495|
|    min|              0.59|
|    max|              21.0|
+-------+------------------+



In [12]:
test_data.describe().show()

+-------+------------------+
|summary|              crew|
+-------+------------------+
|  count|                38|
|   mean| 7.846052631578945|
| stddev|3.3244268268206443|
|    min|              0.88|
|    max|              13.0|
+-------+------------------+



## Fitting and Predicting with Linear Regression model

In [13]:
from pyspark.ml.regression import LinearRegression
model = LinearRegression(featuresCol = 'features', labelCol = 'crew').fit(train_data)
test_results = model.evaluate(test_data)

In [14]:
test_data.describe().show()

+-------+------------------+
|summary|              crew|
+-------+------------------+
|  count|                38|
|   mean| 7.846052631578945|
| stddev|3.3244268268206443|
|    min|              0.88|
|    max|              13.0|
+-------+------------------+



In [15]:
# RMSE 
test_results.rootMeanSquaredError

0.9699431593039742

In [16]:
# R2 Score
test_results.r2

0.9125739443607124